In [1]:
import tensorflow as tf  # Import TensorFlow library for deep learning
from tensorflow.keras.models import Sequential  # Sequential model for building neural networks layer by layer
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D  # Import necessary layers
from tensorflow.keras.callbacks import TensorBoard
import pickle  # Import the pickle library for loading data
import numpy as np  # Import NumPy for numerical operations
import time  # Import time for timestamp generation
import os  # Import os for handling file paths

# Limit CPU usage
os.environ["OMP_NUM_THREADS"] = "2"        # Set the number of OpenMP threads
os.environ["TF_NUM_INTRAOP_THREADS"] = "2" # Number of threads used for intra-op parallelism
os.environ["TF_NUM_INTEROP_THREADS"] = "1" # Number of threads used for inter-op parallelism

# Create a unique name for the TensorBoard logs using the current timestamp
NAME = "Cats-vs-dog-cnn-64x2-without-dense-{}".format(int(time.time()))

# Initialize TensorBoard callback with the correct log directory
log_dir = os.path.join("logs", NAME)
tensorboard = TensorBoard(log_dir=log_dir)

# Load preprocessed image data and labels from pickle files
X = pickle.load(open("X.pickle", "rb"))  # Load feature data (images)
y = pickle.load(open("y.pickle", "rb"))  # Load labels (categories)

# Normalize the pixel values of images to a range of 0 to 1
X = X / 255.0  

# Convert y to a NumPy array to ensure it's in the correct format for training
y = np.array(y)

# Verify that the number of samples in X matches the number of labels in y
assert X.shape[0] == y.shape[0], "Number of samples in X and y must match."

# Configure TensorFlow to use GPU appropriately
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize a Sequential model
model = Sequential()

# Add convolutional layers and pooling layers
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers to feed into the dense layers
model.add(Flatten())

# Add a fully connected (dense) layer
# model.add(Dense(64))
# model.add(Activation("relu"))

# Output layer for binary classification
model.add(Dense(1))
model.add(Activation("sigmoid"))

# Compile the model with specified parameters
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

# Train the model using the training data and TensorBoard callback for logging
model.fit(X, y, batch_size=32, epochs=20, validation_split=0.3, callbacks=[tensorboard])

Epoch 1/20
1638/1638 [==============================] - 231s 140ms/step - loss: 0.5427 - accuracy: 0.7214 - val_loss: 0.4610 - val_accuracy: 0.7862
Epoch 2/20
1638/1638 [==============================] - 228s 139ms/step - loss: 0.4376 - accuracy: 0.7979 - val_loss: 0.4408 - val_accuracy: 0.7910
Epoch 3/20
1638/1638 [==============================] - 227s 138ms/step - loss: 0.3802 - accuracy: 0.8297 - val_loss: 0.3584 - val_accuracy: 0.8360
Epoch 4/20
1638/1638 [==============================] - 227s 139ms/step - loss: 0.3364 - accuracy: 0.8520 - val_loss: 0.3024 - val_accuracy: 0.8744
Epoch 5/20
1638/1638 [==============================] - 225s 137ms/step - loss: 0.2993 - accuracy: 0.8713 - val_loss: 0.2935 - val_accuracy: 0.8739
Epoch 6/20
1638/1638 [==============================] - 226s 138ms/step - loss: 0.2677 - accuracy: 0.8865 - val_loss: 0.2309 - val_accuracy: 0.9046
Epoch 7/20
1638/1638 [==============================] - 231s 141ms/step - loss: 0.2369 - accuracy: 0.9013 - val_